In [7]:
import numpy as np 
import matplotlib.pyplot as plt 
from keras.utils import to_categorical
import nltk
from nltk.tag import hmm
from nltk.probability import LaplaceProbDist
import random

from finger_letter_mapping import letter_finger, finger_letter

In [24]:
f1 = open('training.txt', 'r')
f2 = open('testing.txt', 'r')
training = f1.read().lower().splitlines()
testing = f2.read().lower().splitlines()
f1.close()
f2.close()

In [25]:
print('Letter space (hidden states):\n ', letter_finger.keys())
print('Finger space (observed states): \n', finger_letter.keys())

Letter space (hidden states):
  dict_keys(['q', 'a', 'z', 'w', 's', 'x', 'e', 'd', 'c', 'r', 'f', 'v', 't', 'g', 'b', 'p', 'o', 'l', 'i', 'k', 'u', 'j', 'm', 'y', 'h', 'n', ' '])
Finger space (observed states): 
 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8])


In [26]:
print(training[0])

conversation is interactive communication between two or more people. the development of conversational skills and etiquette is an important part of socialization. the development of conversational skills in a new language is a frequent focus of language teaching and learning. conversation analysis is a branch of sociology which studies the structure and organization of human interaction, with a more specific focus on conversational interaction. no generally accepted definition of conversation exists, beyond the fact that a conversation involves at least two people talking together. consequently, the term is often defined by what it is not. a ritualized exchange such as a mutual greeting is not a conversation, and an interaction that includes a marked status differential (such as a boss giving orders) is also not a conversation. an interaction with a tightly focused topic or purpose is also generally not considered a conversation. summarizing these properties, one authority writes that

In [27]:
def text_to_finger(data): 
    output = [] 
    for row in data:
        output.append(np.array([letter_finger[c] for c in row.lower() if c in letter_finger.keys()]))
    return np.array(output)
def text_to_label(data): 
    output = []
    for row in data: 
        output.append(np.array(list(filter(lambda c: c in letter_finger.keys(), row))))
    return np.array(output)

In [28]:
mapped_training = text_to_finger(training)
mapped_testing = text_to_finger(testing)
labels_training = text_to_label(training)
labels_testing = text_to_label(testing)

# list(map(lambda c: letter_finger[c] if c in letter_finger.keys() else None, sample_text.lower()))
print('reality check:', mapped_training.size, len(labels_training))
print(mapped_training[0].size, labels_training[0].size)

reality check: 51 51
1124 1124


In [29]:
def finger_to_onehot(data): 
    return np.array([to_categorical(row) for row in data])

In [30]:
onehot_training = finger_to_onehot(mapped_training)
onehot_testing = finger_to_onehot(mapped_testing)

In [31]:
# TO DO: add noise to one hot encoding 

# to turn one hot back to label: np.argmax(mapped_one_hot[0])
# mapped_noise = [letter + NOISE for letter in mapped_one_hot]
# for letter in mapped_one_hot: 
    

turn finger mapping back to text using HMM

In [32]:
trainer = hmm.HiddenMarkovModelTrainer(states = letter_finger.keys(), symbols = finger_letter.keys())
# symbols: observations ; states: hidden states

In [33]:
est = LaplaceProbDist

In [34]:
def make_seq(finger, labels): 
    out = []
    for i in range(len(finger)): 
        out.append([(ss, target) for ss, target in zip(finger[i], labels[i])])
    return out

In [35]:
seq = make_seq(mapped_training, labels_training)
test_seq = make_seq(mapped_testing, labels_testing)

In [36]:
tagger = trainer.train_supervised(seq, estimator=est)

In [37]:
tagger.test(test_seq)

accuracy over 2716 tokens: 78.39


In [39]:
tagger.tag([letter_finger[c] for c in 'temptation'])

[(3, 't'),
 (2, 'e'),
 (8, 'm'),
 (5, 'p'),
 (3, 'r'),
 (0, 'a'),
 (3, 't'),
 (7, 'i'),
 (6, 'o'),
 (8, 'n')]